# 🗺️ AI-Venture: GenAI-Powered Travel Companion!
***
An intelligent trip planner that generates personalized travel itineraries, recommends accommodations and experiences, and answers user queries — enhanced with Google Gemini.

## ✅ <u>Problem Statement<u>
Traveling should be fun—not stressful. But planning trips is often difficult without a tour guide or broker, and hiring one can be expensive.  
Searching online for motels, food, and attractions is time-consuming and may not match your preferences. ** ***drumrolls please*** ** AI-Venture's here to save the day!

## 💡 <u>Solution<u>
**AI-Venture** is an AI-powered travel planner built using **Gemini API**. It helps users plan:
- Personalized itineraries
- Stay within a specific budget
- Explore great local attractions and food

## 🤖 <u>Gen AI Capabilities Demonstrated<u>
- 🗺️ **Few-Shot Prompting** to guide Gemini to respond like a personalized travel planner.  
- 🧩 **Structured Output (JSON)** — Parsing results for clarity  
- 🧭 **Function Calling** via Gemini for itinerary generation (budget, distance, time constraints).
- 📄 **Retrieval-Augmented Generation (RAG)** to enable document-based Q&A from PDFs (guides, brochures).
- 🔍 **Embeddings + Vector Search (FAISS)** for accurate and fast retrieval from travel docs.

## 📦 Install Dependencies:
***

In [1]:
!pip install -q -U google-generativeai PyMuPDF faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.5 MB/s eta 0:00:00


## 📚 Import Libraries:
***

In [2]:
import google.generativeai as genai
import os
import pandas as pd
import fitz  
import faiss
import json
import numpy as np
import re
from IPython.display import Markdown, display

## 🔐 API Key Setup:
***

In [3]:
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Set your Gemini API key
genai.configure(api_key=GOOGLE_API_KEY)

# Create a model instance
model = genai.GenerativeModel("gemini-2.0-flash")

## Define Dataframes/Database:

We’ll manually define ...

In [4]:
# Sample motels, restaurants, attractions
motels = pd.DataFrame([
    # Paris
    {"name": "Cozy Inn", "location": "Paris", "price_per_night": 45, "rating": 4.2},
    {"name": "Budget Stay", "location": "Paris", "price_per_night": 30, "rating": 3.8},
    {"name": "Chic Rooms", "location": "Paris", "price_per_night": 65, "rating": 4.5},

    # Rome
    {"name": "Rome Comfort Hotel", "location": "Rome", "price_per_night": 50, "rating": 4.0},
    {"name": "Trastevere Budget Inn", "location": "Rome", "price_per_night": 35, "rating": 3.7},
    {"name": "Hotel Bella Roma", "location": "Rome", "price_per_night": 60, "rating": 4.3},

    # Berlin
    {"name": "Berlin Central Stay", "location": "Berlin", "price_per_night": 48, "rating": 4.1},
    {"name": "Budget Lodge Berlin", "location": "Berlin", "price_per_night": 32, "rating": 3.9},
    {"name": "Checkpoint Hostel", "location": "Berlin", "price_per_night": 27, "rating": 3.6},

    # Barcelona
    {"name": "Barcelona Breeze", "location": "Barcelona", "price_per_night": 55, "rating": 4.4},
    {"name": "La Rambla Inn", "location": "Barcelona", "price_per_night": 40, "rating": 4.0},
    {"name": "El Centro Lodge", "location": "Barcelona", "price_per_night": 33, "rating": 3.8}
])

restaurants = pd.DataFrame([
    # Paris
    {"name": "Veggie Delight", "location": "Paris", "type": "Vegetarian", "price_range": "Low"},
    {"name": "Cafe Royale", "location": "Paris", "type": "Cafe", "price_range": "Medium"},
    {"name": "Seafood Heaven", "location": "Paris", "type": "Seafood", "price_range": "High"},

    # Rome
    {"name": "Pasta Fresca", "location": "Rome", "type": "Italian", "price_range": "Medium"},
    {"name": "Vegan Vibe", "location": "Rome", "type": "Vegetarian", "price_range": "Low"},
    {"name": "Trattoria Romano", "location": "Rome", "type": "Traditional", "price_range": "High"},

    # Berlin
    {"name": "Berlin Bites", "location": "Berlin", "type": "Fast Food", "price_range": "Low"},
    {"name": "Wurst Haus", "location": "Berlin", "type": "German", "price_range": "Medium"},
    {"name": "Green Eats", "location": "Berlin", "type": "Vegan", "price_range": "Low"},

    # Barcelona
    {"name": "Tapas Town", "location": "Barcelona", "type": "Tapas", "price_range": "Medium"},
    {"name": "La Veggiera", "location": "Barcelona", "type": "Vegetarian", "price_range": "Low"},
    {"name": "Mar y Sol", "location": "Barcelona", "type": "Seafood", "price_range": "High"}
])

attractions = pd.DataFrame([
    # Paris
    {"name": "Eiffel Tower", "location": "Paris", "type": "Landmark", "duration": "2 hours"},
    {"name": "Louvre Museum", "location": "Paris", "type": "Museum", "duration": "3 hours"},
    {"name": "Seine River Cruise", "location": "Paris", "type": "Experience", "duration": "1.5 hours"},

    # Rome
    {"name": "Colosseum", "location": "Rome", "type": "Landmark", "duration": "2 hours"},
    {"name": "Vatican Museums", "location": "Rome", "type": "Museum", "duration": "3 hours"},
    {"name": "Trevi Fountain", "location": "Rome", "type": "Landmark", "duration": "1 hour"},

    # Berlin
    {"name": "Brandenburg Gate", "location": "Berlin", "type": "Landmark", "duration": "1 hour"},
    {"name": "Pergamon Museum", "location": "Berlin", "type": "Museum", "duration": "2.5 hours"},
    {"name": "Berlin Wall Memorial", "location": "Berlin", "type": "Historical", "duration": "1.5 hours"},

    # Barcelona
    {"name": "Sagrada Familia", "location": "Barcelona", "type": "Landmark", "duration": "2 hours"},
    {"name": "Park Güell", "location": "Barcelona", "type": "Experience", "duration": "1.5 hours"},
    {"name": "Picasso Museum", "location": "Barcelona", "type": "Museum", "duration": "2 hours"}
])

## 🧠 Step 2: Teaching the AI with Examples (Few-Shot Prompting)

We’re going to provide a few example inputs and desired outputs.  
This helps Gemini understand what a “good itinerary” looks like.

We use:
- Example input/output
- User's input at the end
- Ask Gemini to respond **only in JSON**

In [5]:
# Function to build the few-shot prompt for the model
def build_few_shot_prompt(user):
    return f"""
You are a travel planner assistant. Based on the user's input, generate a {user["days"]}-day travel itinerary in JSON format.

Instructions:
- Recommend motels, attractions, and restaurants that match the user's preferences.
- Stay strictly within the total budget (in USD).
- Keep each day's total cost proportional to the total budget.
- Include estimated cost breakdowns (in USD) for:
  - Motel
  - Food
  - Attraction entrance fees (if applicable)
- Mention attraction durations where useful.
- **Do not use tildes (~) anywhere.**
- Respond in clean, readable JSON only (no explanations).
- Follow this structure exactly.

Example:

User: {{
  "name": "Aarav",
  "destination": "Tokyo",
  "days": 2,
  "budget": 150,
  "preferences": {{
    "food": "Vegetarian",
    "interests": ["Museum", "Cultural"]
  }}
}}

Response:
{{
  "day_1": {{
    "attractions": [
      "Tokyo National Museum (Entry approx. $7, Duration: 2 hrs)",
      "Asakusa Shrine (Free, Duration: 1 hr)"
    ],
    "restaurant": "Veggie Sora (Estimated around $14)",
    "motel": "Tokyo Budget Inn (Estimated around $28/night)",
    "total_cost": 70
  }},
  "day_2": {{
    "attractions": [
      "Meiji Shrine (Free, Duration: 1.5 hrs)",
      "Ueno Park (Free, Duration: 1 hr)"
    ],
    "restaurant": "Green Table Tokyo (Estimated around $14)",
    "motel": "Tokyo Budget Inn (Estimated around $28/night)",
    "total_cost": 70
  }},
  "notes": "Remaining $10 can be used for transport or snacks."
}}

Now generate for this user:

User: {json.dumps(user, indent=2)}
Response:
"""

## 🧳 Step 3: Enter Your Travel Preferences

You can change this JSON script below 👇🏼 to create your own trip. Modify:
- `destination`
- `days`
- `budget`
- `food` type
- `interests`

Then run the cells below to generate your own travel plan!

In [6]:
# Example user travel preferences
user_input = {
    "name": "John",
    "destination": "Paris",
    "days": 2,
    "budget": 200,
    "preferences": {
        "food": "Vegetarian",
        "interests": ["Landmark", "Museum"]
    }
}

In [7]:
prompt = build_few_shot_prompt(user_input)

response = model.generate_content(prompt)

# Print raw response
print(response.text)

```json
{
  "day_1": {
    "attractions": [
      "Eiffel Tower (Entry approx. $30, Duration: 2 hrs)",
      "Louvre Museum (Entry approx. $20, Duration: 3 hrs)"
    ],
    "restaurant": "Le Grenier de Notre-Dame (Vegetarian options, Estimated around $25)",
    "motel": "Hotel Parisian (Estimated around $35/night)",
    "total_cost": 110
  },
  "day_2": {
    "attractions": [
      "Arc de Triomphe (Entry approx. $15, Duration: 1 hr)",
      "Sainte-Chapelle (Entry approx. $12, Duration: 1.5 hrs)"
    ],
    "restaurant": "L'Arpege (Vegetarian tasting menu available, Estimated around $30)",
    "motel": "Hotel Parisian (Estimated around $35/night)",
    "total_cost": 92
  },
  "notes": "Consider purchasing a travel pass for transport to save on individual tickets. Prices are estimates and may vary."
}
```


## 🗓️ Step 4: View Your Personalized Itinerary

Gemini replies in JSON format. We’ll parse it and display:
- Attractions
- Motels
- Restaurants
- Estimated cost
- Notes / Tips

In [8]:
# Function to escape markdown-sensitive characters (still useful for other fields)
def clean_for_markdown(text):
    text = text.replace("_", "\\_")
    text = text.replace("~", "\\~")
    text = text.replace("*", "\\*")
    text = text.replace("`", "\\`")
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text)
    return text

try:
    # Remove ```json or ``` if present
    cleaned_text = re.sub(r"^```(?:json)?|```$", "", response.text.strip(), flags=re.MULTILINE).strip()
    cleaned_text = cleaned_text.replace("“", "\"").replace("”", "\"").replace("’", "'")

    itinerary = json.loads(cleaned_text)

    display(Markdown("## ✈️| Travel Itinerary"))

    total_cost = 0

    for day, plan in itinerary.items():
        if day.startswith("day"):
            # Replace underscores with spaces and title-case the label
            day_label = day.replace("_", " ").title()  # "day_1" → "Day 1"
            display(Markdown(f"### 🗓️ {day_label}"))

            # Clean each element to escape markdown characters
            parsed_attractions = [clean_for_markdown(attraction) for attraction in plan["attractions"]]
            parsed_restaurant = clean_for_markdown(plan["restaurant"])
            parsed_motel = clean_for_markdown(plan["motel"])

            # Display
            display(Markdown(f"- 🗽 **Attractions:** {', '.join(parsed_attractions)}"))
            display(Markdown(f"- 🍽️ **Restaurant:** {parsed_restaurant}"))
            display(Markdown(f"- 🛏️ **Motel:** {parsed_motel}"))
            display(Markdown(f"- 💰 **Estimated Cost:** ${plan['total_cost']}"))
            total_cost += plan["total_cost"]

        elif day == "notes":
            display(Markdown(f"**📝 Notes:** {clean_for_markdown(plan)}"))

    # Final budget summary
    display(Markdown(f"### 💸 **Total Estimated Trip Cost:** ${total_cost}"))
    remaining = user_input["budget"] - total_cost

    if remaining > 0:
        display(Markdown(f"✅ **Remaining Budget:** ${remaining}"))
    elif remaining < 0:
        display(Markdown(f"⚠️ **Over Budget by:** ${-remaining}"))
    else:
        display(Markdown("📎 **Used the entire budget perfectly!**"))

except Exception as e:
    print("❌ Could not parse response as JSON:", e)
    print("Raw response:\n", response.text)

## ✈️| Travel Itinerary

### 🗓️ Day 1

- 🗽 **Attractions:** Eiffel Tower (Entry approx. $30, Duration: 2 hrs), Louvre Museum (Entry approx. $20, Duration: 3 hrs)

- 🍽️ **Restaurant:** Le Grenier de Notre-Dame (Vegetarian options, Estimated around $25)

- 🛏️ **Motel:** Hotel Parisian (Estimated around $35/night)

...

- 🛏️ **Motel:** Hotel Parisian (Estimated around $35/night)

- 💰 **Estimated Cost:** $92

**📝 Notes:** Consider purchasing a travel pass for transport to save on individual tickets. Prices are estimates and may vary.

### 💸 **Total Estimated Trip Cost:** $202

⚠️ **Over Budget by:** $2

## 🔧 Step 5: Function Calling with Gemini

Gemini can now *intelligently decide* which Python function to call, based on user queries.

We’ll define:
- 🔍 A `find_motels` function for matching budget, location
- 🍽️ A `find_restaurants` function based on food type
- 🎢 A `find_attractions` function based on interests

Let’s empower Gemini to become a truly interactive travel assistant!

In [9]:
# Functions Gemini can call

def find_motels(location: str, max_price: float):
    return motels[(motels["location"] == location) & (motels["price_per_night"] <= max_price)].to_dict(orient="records")

def find_restaurants(location: str, food_type: str):
    return restaurants[(restaurants["location"] == location) & (restaurants["type"].str.contains(food_type, case=False))].to_dict(orient="records")

def find_attractions(location: str, interest_type: str):
    return attractions[(attractions["location"] == location) & (attractions["type"].str.contains(interest_type, case=False))].to_dict(orient="records")

## 📄 Step 6: Define the Function Schema for Gemini

Here we define the function names, descriptions, and parameters Gemini can use to choose what to call. These are sent to Gemini as metadata.

In [10]:
print("Type of attractions:", type(attractions))

Type of attractions: <class 'pandas.core.frame.DataFrame'>


In [11]:
functions = [
    {
        "name": "find_motels",
        "description": "Find budget motels in a specific location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"},
                "max_price": {"type": "number"}
            },
            "required": ["location", "max_price"]
        }
    },
    {
        "name": "find_restaurants",
        "description": "Find restaurants by location and food type.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"},
                "food_type": {"type": "string"}
            },
            "required": ["location", "food_type"]
        }
    },
    {
        "name": "find_attractions",
        "description": "Find attractions by location and interest type.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"},
                "interest_type": {"type": "string"}
            },
            "required": ["location", "interest_type"]
        }
    }
]

## 🧠 Step 7: Load Gemini Model with Function Calling Enabled

In [12]:
model_fc = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[{"function_declarations": functions}]
)

## 💬 Step 8: Ask Your Custom Travel Query!

Try asking things like:
- “Show me motels in Paris under $50”
- “Find vegetarian restaurants in Paris”
- “What museums can I visit in Paris?”

Gemini will choose the right function automatically!

### 🔧 How to Make the Code Interactive

To make the query interactive, uncomment the line containing `input()` below. This will prompt you to enter your own travel query when you run the cell.

By default, it’s set to a sample query. If you want to test it with your own query, simply uncomment the line and run the cell again.

Example:
```python
### user_query = input("💬 Enter your travel query: ")  # Uncomment this line for interactive input

⚠️ **Note for Kaggle Users: Save Without Crashes**

This cell block below uses `input()` and Gemini chat, which will throw an error if you click **"Save & Run All"** in Kaggle.

To avoid this, use the `RUN_GEMINI_QUERY` flag below:

- Set `RUN_GEMINI_QUERY = False` to skip chat logic when saving.
- Set `RUN_GEMINI_QUERY = True` to run and test the chatbot.

Only run the chatbot **manually**, not during "Save & Run All".

In [13]:
RUN_GEMINI_QUERY = False  # 👈 Change to True to test Gemini chat

# 🔄 Gemini Chat Start (Guarded by flag)
if RUN_GEMINI_QUERY:
    chat = model_fc.start_chat()

    # 🔄 Interactive Query
    user_query = "What museums can I visit in Paris?"  # Default query
    # user_query = input("💬 Enter your travel query: ")  # Uncomment for interactive input

    # Send it to Gemini
    response = chat.send_message(user_query)

    if response.candidates[0].content.parts:
        result = response.candidates[0].content.parts[0].function_call
        print("🧠 Function to Call:", result.name)

        # Convert args to dict (Safe fix for notebook)
        try:
            args_dict = {k: v for k, v in result.args.items()}
            print("📦 Arguments:", args_dict)
        except Exception as e:
            print(f"❌ Error converting arguments: {e}")
            args_dict = {}

        # Call the corresponding Python function
        if result.name == "find_motels":
            output = find_motels(**args_dict)
        elif result.name == "find_restaurants":
            output = find_restaurants(**args_dict)
        elif result.name == "find_attractions":
            output = find_attractions(**args_dict)
        else:
            output = []

        # Display results
        if output:
            print("🔎 Gemini's Results:")
            for i, entry in enumerate(output, 1):
                print(f"\nResult #{i}")
                for k, v in entry.items():
                    print(f"- {k.capitalize()}: {v}")
        else:
            print("🤷 No matching results found.")
    else:
        print("❌ Gemini didn’t suggest a function to call.")